In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from scipy.linalg import norm

In [2]:
df = pd.read_csv('sample_data/california_housing_train.csv')
df1=pd.read_csv('sample_data/california_housing_test.csv')

In [3]:
y = df['median_house_value'].values
A=df.drop(['median_house_value'],axis =1).values

In [4]:

A=A[:500]
y=y[:500]

mA = A.mean(axis=0)
sA = A.std(axis=0)
A = (A-mA)/sA
m = y.mean()
y = y-m

In [5]:
class RegPb(object):
    '''                                                                   
        A class for regression problems with linear models.
        
        Attributes:
            A: Data matrix (features)
            y: Data vector (labels)
            n,d: Dimensions of A
            loss: Loss function to be considered in the regression
                'l2': Least-squares loss
                'logit': Logistic loss
            lbda: Regularization parameter
    '''
   
    # Instantiate the class
    def __init__(self, A, y,lbda=0):
        self.A = A
        self.y = y
        self.n, self.p = A.shape
       
        self.lbda = lbda
        
    
    # Objective value
    def f(self, x):
      return np.linalg.norm(self.A.dot(x) - self.y) ** 2 / (2. ) + self.lbda * norm(x) ** 2 / 2.
       
    # Partial objective value
    def f_i(self, i, x):
       
      return np.linalg.norm(self.A[i].dot(x) - self.y[i]) ** 2 / (2.) + self.lbda * norm(x) ** 2 / 2.
 
    
    # Full gradient computation
    def grad(self, x):
        
    
        return self.A.T.dot(self.A.dot(x) - self.y)  + self.lbda * x
       
    
    # Partial gradient
    def grad_i(self,i,x):
        a_i = self.A[i]
        
        
        return (a_i.dot(x) - self.y[i]) * a_i + self.lbda*x
      
    def fl1(self,x):
        return self.f(x)+norm(x,ord=1)

    # maximum learning rate  for the gradient
    def tau(self):
        C = A.transpose().dot(A)
        L = norm(C+self.lbda*np.eye(self.p), ord=2) ** 2  
       
        return 1/L
    # optimal learning rate
    def tau_opt(self):
      C = A.transpose().dot(A)
      tau= 2/( norm(C+self.lbda*np.eye(self.p), ord=2) ** 2+ norm(C+self.lbda*np.eye(self.p), ord=-2) ** 2)
      return tau

In [10]:
def gradientdescent(prob,x0,x_min,iter,tau): ## 
  
  s=x0
  L=[]
  val=0
  for i in range(iter):
    
    
    g=prob.grad(s)
    
    s=s - tau*g
    
    val=prob.f(s)
    L.append(val)
  return np.array(L),s


In [14]:
p=8

In [20]:
import random
def coordinategradientdescent(prob,x0,xtarget,tau_opt,step0=1, n_iter=1000,nb=1,with_replace=False,GD=False): 
   
    # Initial step: Compute and plot some initial quantities

    # objective history
    objvals = []
    
    # iterates distance to the minimum history
    
    with_replace=False
    # Lipschitz constant
    L = norm(A,ord=2)**2
    
    # Number of samples
    n = 500
    
    # Initial value of current iterate  
    x = x0.copy()
   

    # Initialize iteration counter
    k=0
    
    # Current objective
    obj = prob.f(x)
    objvals.append(obj);
    # Current distance to the optimum
    
    
    
  
    # Main loop
    while (k < n_iter):
        # Draw the batch indices
        # Batch gradient
        ik=random.sample(range(0,p ), nb)
        
        # Stochastic gradient calculation
        
        sg = np.zeros(p)

        for j in range(nb):
            sg[ik[j]] = prob.grad(x)[ik[j]]
            
        
        
        
        
        x = x - tau_opt * sg
            
        
      
        
        obj =prob.f(x)
       
        
        k += 1
        
        if k*nb % 8==0:
            
            objvals.append(obj)
        
              
    
    # Outputs
    x_output = x
    
    return x_output, np.array(objvals), 


In [ ]:
n_epochs=60
x0=np.zeros(8)
prob=RegPb(A,y)
tau_opt=prob.tau_opt()
L5=gradientdescent(prob,x0,0,n_epochs,tau_opt)[0]
R1=coordinategradientdescent(prob,x0,0,tau_opt/2,0, n_epochs*500,7)[1]
plt.figure(figsize=(7, 5))
plt.semilogy(R1, label="Coordinate Block Descent", lw=2)

plt.semilogy(L5, label="Gradient Descent", lw=2)

plt.legend()
plt.show()

In [ ]:
import random
# Stochastic gradient implementation
def coordinatestochastic(prob,x0,xtarget,stepchoice=0,step0=1, n_iter=1000,nb=1,nb1=1,with_replace=False,GD=False): 
   
    # Initial step: Compute and plot some initial quantities

    # objective history
    objvals = []
    
    # iterates distance to the minimum history
    
    with_replace=False
    # Lipschitz constant
    L = norm(A,ord=2)**2
    
    # Number of samples
    n = 500
    
    # Initial value of current iterate  
    x = x0.copy()
   

    # Initialize iteration counter
    k=0
    
    # Current objective
    obj = prob.f(x)
    objvals.append(obj);
    # Current distance to the optimum
    
    
    
  
    # Main loop
    while (k < n_iter):
        # Draw the batch indices
        ik = np.random.choice(500,nb,replace=with_replace)# Batch gradient
        jk=random.sample(range(0,p ), nb1)
        # Stochastic gradient calculation
        
        sg = np.zeros(p)
        for j in range(nb):
            gi = prob.grad_i(ik[j],x)
            sg = sg + gi
        sg = (1/nb)*sg
        tg=np.zeros(p)
        for j in range(nb1):
          tg[jk[j]]=sg[jk[j]]
          
        
        if stepchoice==0:
            x = x - (step0/L) * tg
        elif stepchoice>0:
            sk = float(step0/((k+1)**0.5))
            
            x = x - sk * tg
        
      
        
        obj =prob.f(x)
       
        
        k += 1
        # Plot quantities of interest at the end of every epoch only
        if (k*nb*nb1) % 8*n == 0:
            objvals.append(obj)
            
    
    # Plot quantities of interest for the last iterate (if needed)
    if (k*nb) % n > 0:
        objvals.append(obj)
        
              
    
    # Outputs
    x_output = x
    
    return x_output, np.array(objvals), 
